In [ ]:
import os

import sys
print(sys.version)

import glob
import h5py
import cv2
import keras
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, Input, Flatten, Lambda, Reshape, Cropping2D
from keras.models import Model
from keras.datasets import mnist
import numpy as np
from keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Model
import scipy.stats as stats
import PIL
from PIL import Image

import dask
import dask.array as da
from dask.distributed import Client
from dask_image.imread import imread


In [ ]:
# Increase the maximum image pixels limit
PIL.Image.MAX_IMAGE_PIXELS = None

# batch = 200
batch = 2

train_dir = 'train_folder'
test_dir = 'test_folder'

train_samples = len(glob.glob(os.path.join(train_dir, '**', '*.jpg'), recursive=True))
test_samples = len(glob.glob(os.path.join(test_dir, '**', '*.jpg'), recursive=True))

print(f"Training samples: {train_samples}")
print(f"Testing samples: {test_samples}")

# img_height, img_width = 15216, 21397 # Resize to smaller dimensions
# img_height, img_width = 512, 512 # Resize to smaller dimensions
# img_height, img_width = 32, 64 # Resize to smaller dimensions
# img_height, img_width = 3044, 4280 # Resize to smaller dimensions
img_height, img_width = 1522, 2140 # Resize to smaller dimensions



num_channels = 3 # RGB

datagen = ImageDataGenerator(rescale=1./255)

# Load training set in batches
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=batch,
    class_mode=None,
    shuffle=True,
    seed=42)

# Load test set in batches
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=batch,
    class_mode=None,
    shuffle=True,
    seed=42)

# Example of loading a batch
x_train_small = next(train_generator)
x_test_small = next(test_generator)

print("Train batch shape:", x_train_small.shape)
print("Test batch shape:", x_test_small.shape)

img_shape = (img_height, img_width, num_channels)

In [ ]:
anomaly_dir = 'anomaly_folder'

anomaly_samples = len(glob.glob(os.path.join(anomaly_dir, '**', '*.jpg'), recursive=True))
print(f"Anomaly samples: {anomaly_samples}")

anomaly_generator = datagen.flow_from_directory(
    anomaly_dir,
    target_size=(img_height, img_width),
    color_mode='rgb',
    # batch_size=99999,
    batch_size=batch,
    class_mode=None,
    shuffle=True,
    seed=42)

anomaly_data = next(anomaly_generator)
anomaly_test_data = anomaly_data.copy()

In [ ]:
print(type(x_train_small))
print(type(anomaly_data))
print(len(x_train_small))
print(len(x_test_small))
print(len(anomaly_data))

In [ ]:
print("x_train_small shape:", x_train_small.shape)
print("train_samples:", train_samples)

n = []
a = 0
for j in range(0, 10):
    n.append(random.randint(0, len(x_train_small) - 1))  # Use -1 to avoid index out of range
print(n)

print('\nFew of the training data samples are:')
plt.figure(figsize=(20, 4))
for i in n:
    a = a + 1
    ax = plt.subplot(2, 10, a)
    plt.imshow(x_train_small[i])  # No reshape needed for RGB images
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()


In [ ]:
f = []
b = 0
for j in range(0, 10):
    f.append(random.randint(0, len(anomaly_data)-1))
print(f)

print('\nFew of the Anomaly data samples are:')
plt.figure(figsize=(20, 4))
             
for i in f:
    b = b + 1
    ax = plt.subplot(2, 10, b)
    plt.imshow(anomaly_data[i])  # No reshape needed for RGB images
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
input_img = keras.Input(shape = img_shape)

x = keras.layers.Conv2D(10, (3, 3), activation='relu', padding='same')(input_img)
x = keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = keras.layers.Conv2D(10, (3, 3), activation='relu', padding='same')(x)
x = keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = keras.layers.MaxPooling2D((2, 2), padding='same')(x)

# x = keras.layers.Conv2D(10, (3, 3), activation='relu', padding='same')(encoded)
# x = keras.layers.UpSampling2D((2, 2))(x)
# x = keras.layers.Conv2D(10, (3, 3), activation='relu', padding='same')(x)
# x = keras.layers.UpSampling2D((2, 2))(x)
# x = keras.layers.Conv2D(16, (3, 3), activation='relu')(x)
# x = keras.layers.UpSampling2D((2, 2))(x)
# decoded = keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)


x = keras.layers.Conv2D(10, (3, 3), activation='relu', padding='same')(encoded)
x = keras.layers.UpSampling2D((2, 2))(x)
x = keras.layers.Conv2D(10, (3, 3), activation='relu', padding='same')(x)
x = keras.layers.UpSampling2D((2, 2))(x)
x = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = keras.layers.UpSampling2D((2, 2))(x)
# decoded = keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
x = keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
decoded = Cropping2D(((0,6), (0,4)))(x)  # Trim 6 rows and 4 columns

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

# Evaluation model where output is the loss
loss_layer = Lambda(lambda x: K.mean(K.square(x[0] - x[1])), name='loss_layer', output_shape=lambda s: s[0])
# Define the model
model_evalloss = Model(inputs=[input_img, input_img], outputs=loss_layer([input_img, decoded]))
# model_evalloss.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model_evalloss.summary()


In [ ]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# x_train_small = x_train_small.astype('float16')/255.0
# x_test_small = x_test_small.astype('float16')/255.0

# # Example: Resize each image in the dataset to 128x128
# x_train_small_resized = np.array([np.array(Image.open(img).resize((128, 128))) for img in x_train_small])
# x_test_small_resized = np.array([np.array(Image.open(img).resize((128, 128))) for img in x_test_small])


# print(f"Training data size (GB): {x_train_small.nbytes / (1024**3):.2f} GB")
# print("Shape:", x_train_small.shape)

# print(f"Testing data size (GB): {x_test_small.nbytes / (1024**3):.2f} GB")
# print("Shape:", x_test_small.shape)

# print(tf.config.list_physical_devices('GPU'))

# gpu_devices = tf.config.list_physical_devices('GPU')  # Non-experimental method
# for device in gpu_devices:
#     print(tf.config.experimental.get_memory_info("GPU:0"))  # Use "GPU:0" directly


def data_generator(x_data, batch_size):
    while True:  # For infinite training
        for i in range(0, len(x_data), batch_size):
            x_batch = x_data[i:i+batch_size]
            yield x_batch, x_batch  # for autoencoder: input = target

# batch = 1  # or 2, depending on memory
train_gen = tf.data.Dataset.from_generator(
    lambda: data_generator(x_train_small, batch),
    output_signature=(
        tf.TensorSpec(shape=(None, 1522, 2140, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1522, 2140, 3), dtype=tf.float32),
    )
)

val_gen = tf.data.Dataset.from_generator(
    lambda: data_generator(x_test_small, batch),
    output_signature=(
        tf.TensorSpec(shape=(None, 1522, 2140, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 1522, 2140, 3), dtype=tf.float32),
    )
)

a = autoencoder.fit(
    train_gen,
    # epochs=25,
    epochs=20000,
    steps_per_epoch=len(x_train_small) // batch,
    validation_data=val_gen,
    validation_steps=len(x_test_small) // batch,
    # callbacks=[GPUMemoryCallback()]
)

In [ ]:
results = autoencoder.evaluate(x_test_small, x_test_small, verbose=0)
print ("Test MSE loss:", results)

results_anom = autoencoder.evaluate(anomaly_data, anomaly_data, verbose=0)
print ("Anomaly MSE loss:", results_anom)




In [ ]:
#Draw a test image
im_test = autoencoder.predict(x_test_small[:1])
plt.imshow(im_test[0])

In [ ]:
# Draw an anomaly image
im = autoencoder.predict(anomaly_data[:1])
plt.imshow(im[0])

In [ ]:
decoded_imgs = autoencoder.predict(x_test_small)
decoded_imgs_anom = autoencoder.predict(anomaly_data)

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
loss = a.history['loss']
val_loss = a.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, '-o', label='Training loss')
plt.plot(epochs, val_loss, '-o', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
print("The orignal images are:")
for i in range(0, min(n, len(x_test_small))):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_small[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

print("\nThe reconstructed images are:")
plt.figure(figsize=(20, 4))
for i in range(0, min(n, len(decoded_imgs))):
    ax = plt.subplot(2, n, i + n + 1)
    plt.imshow(decoded_imgs[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()